In [ ]:
# حذف نسخه‌های قبلی
!pip uninstall transformers sentence-transformers torch numpy pandas datasets hazm tokenizers -y

# نصب نسخه‌های سازگار با هم
!pip install numpy==1.23.5
!pip install pandas==1.5.3
!pip install torch==2.0.1 torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip install transformers==4.33.2
!pip install datasets==2.14.0
!pip install hazm


# 📦 تست import ها
try:
    import pandas as pd
    import torch
    import numpy as np
    from hazm import Normalizer
    from transformers import (
        AutoTokenizer,
        AutoModelForSequenceClassification,
        TrainingArguments,
        Trainer
    )
    from datasets import Dataset

    print("✅ همه کتابخانه‌ها با موفقیت import شدند!")
    print(f"🔹 Transformers version: {torch.__version__}")
    print(f"🔹 Numpy version: {np.__version__}")
    print(f"🔹 Pandas version: {pd.__version__}")

except Exception as e:
    print(f"❌ خطا: {e}")

ImportError: cannot import name 'PreTrainedModel' from 'transformers' (/usr/local/lib/python3.11/dist-packages/transformers/__init__.py)

In [ ]:
# لینک‌ها
url1 = "https://raw.githubusercontent.com/davardoust/PHICAD/main/PHICAD-part1.csv"
url2 = "https://raw.githubusercontent.com/davardoust/PHICAD/main/PHICAD-part2.csv"

# 📥 خواندن فایل‌ها با تنظیمات دقیق
df1 = pd.read_csv(url1, sep="\t", header=0)
df2 = pd.read_csv(url2, sep="\t", header=0)

# 🔗 ادغام
df = pd.concat([df1, df2], ignore_index=True)

# حذف ستون‌های اضافه‌ای که اشتباه وارد شدن
expected_columns = ['comment_normalized', 'hate', 'spam', 'obscene', 'class']
df = df[expected_columns]

# نمایش اولیه
print(f"🔢 تعداد کل داده‌ها: {len(df)}")
print("🧾 ستون‌ها:", df.columns.tolist())
print(df.head(3))

In [ ]:
from sklearn.preprocessing import LabelEncoder

# ستون کلاس رو encode می‌کنیم (مثلاً clean → 0، spam → 1 ...)
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["class"])

# نمایش کلاس‌ها و مقدار عددی‌شون
label_map = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("🔢 کلاس‌ها:", label_map)

# تبدیل pandas → HuggingFace Dataset
dataset = Dataset.from_pandas(df[["comment_normalized", "label"]])
print("✅ Dataset آماده است!")


In [ ]:
normalizer = Normalizer()
model_name = "HooshvareLab/bert-base-parsbert-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_data(texts):
    norm = [normalizer.normalize(t) for t in texts]
    return tokenizer(norm, truncation=True, padding=True, max_length=128)

encodings = preprocess_data(df['text'].tolist())
labels = df['label'].tolist()

In [ ]:
dataset = Dataset.from_dict({
    'input_ids': encodings['input_ids'],
    'attention_mask': encodings['attention_mask'],
    'labels': labels
})

splits = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = splits['train']
eval_dataset = splits['test']

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

In [ ]:
training_args = TrainingArguments(
    output_dir="./phicad_results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    logging_steps=100,
    logging_dir='./phicad_logs'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

In [ ]:
trainer.train()

In [ ]:
metrics = trainer.evaluate()
print(metrics)

model.save_pretrained("phicad_model")
tokenizer.save_pretrained("phicad_model")

In [ ]:
def is_inappropriate(text):
    t = normalizer.normalize(text)
    inputs = tokenizer(t, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        logits = model(**inputs).logits
    return torch.argmax(logits, dim=1).item() == 1

comments = [
    "لعنت به این وضعیت خرابات!",
    "خیلی ممنون از خدمات شما"
]
for c in comments:
    print(c, "→", "نامناسب" if is_inappropriate(c) else "مناسب")